In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17923092
paper_name = 'mcclellan_frydman_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/mmc2.xlsx', sheet_name='Sheet1', skiprows=2)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4125 x 5


In [10]:
original_data.head()

,hom 30 °C entire ranked list,Average Rv,Unnamed: 2,hom 37 °C entire ranked list,Average Rv.1
0,STI1,0.108,NaN,PDR1,0.13884
1,PDR1,0.158,NaN,PTR2,0.15225
2,COG5,0.164,NaN,stp22,0.18005
3,CYK3,0.164,NaN,VPS28,0.21785
4,PTR2,0.174,NaN,SNF8,0.22166


In [16]:
original_data_list = [original_data.iloc[:,[0,1]].copy(), original_data.iloc[:,[3,4]].copy()]

In [21]:
original_data_list2 = []
for d in original_data_list:
    d['gene'] = d.iloc[:,0].astype(str)
    d['gene'] = clean_genename(d['gene'])
    d['orf'] = translate_sc(d['gene'], to='orf')
    t = looks_like_orf(d['orf'])
    print(d.loc[~t,])
    d = d.loc[t,:]
    
    d['data'] = pd.to_numeric(d.iloc[:,1], errors='coerce')
    d.set_index('orf', inplace=True)
    d = d[['data']].copy()
    d = d.groupby(d.index).mean()
    print(d.shape)
    original_data_list2.append(d)

            hom 30 °C entire ranked list  Average Rv   gene    orf
index_input                                                       
53                            HXT12            0.610  HXT12  HXT12
126                                  FMO       0.774    FMO    FMO
572                           HSN1             1.002   HSN1   HSN1
1084                          PMN1             1.076   PMN1   PMN1
1272                          GPE1             1.096   GPE1   GPE1
...                                  ...         ...    ...    ...
4120                                 NaN         NaN    NAN    NAN
4121                                 NaN         NaN    NAN    NAN
4122                                 NaN         NaN    NAN    NAN
4123                                 NaN         NaN    NAN    NAN
4124                                 NaN         NaN    NAN    NAN

[355 rows x 4 columns]
(3744, 1)


/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


            hom 37 °C entire ranked list  Average Rv.1   gene    orf
index_input                                                         
48                            HXT12            0.42990  HXT12  HXT12
100                           AMI3             0.53907   AMI3   AMI3
238                           GPE1             0.67849   GPE1   GPE1
289                           TOS9             0.72538   TOS9   TOS9
438                           GPE2             0.80990   GPE2   GPE2
720                           HSN1             0.88382   HSN1   HSN1
1106                          EFR4             0.95151   EFR4   EFR4
1609                          BZR1             1.01160   BZR1   BZR1
1741                          SRI1             1.02690   SRI1   SRI1
1890                          TOS7             1.04520   TOS7   TOS7
2010                          CRS5             1.05900   CRS5   CRS5
2156                          FLO8             1.07920   FLO8   FLO8
2613                          KIM1

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [22]:
original_data = pd.concat(original_data_list2, axis=1)

In [23]:
original_data.head()

,data,data
YAL002W,0.450,0.7616
YAL004W,1.162,1.3888
YAL005C,1.270,1.2287
YAL007C,1.134,1.1995
YAL008W,1.114,1.1079


In [24]:
original_data.index.name='orf'

In [25]:
original_data.shape

(4105, 2)

# Prepare the final dataset

In [26]:
data = original_data.copy()

In [27]:
dataset_ids = [11785,11786]
datasets = datasets.reindex(index=dataset_ids)

In [28]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [29]:
data.head()

dataset_id,11785,11786
data_type,value,value
orf,,
YAL002W,0.450,0.7616
YAL004W,1.162,1.3888
YAL005C,1.270,1.2287
YAL007C,1.134,1.1995
YAL008W,1.114,1.1079


## Subset to the genes currently in SGD

In [30]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 16


In [31]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,11785,11786
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.450,0.7616
1863,YAL004W,1.162,1.3888
4,YAL005C,1.270,1.2287
5,YAL007C,1.134,1.1995
6,YAL008W,1.114,1.1079


# Normalize

In [32]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [33]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [34]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,11785,11786,11785,11786
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.450,0.7616,-3.044697,-0.357136
1863,YAL004W,1.162,1.3888,0.017289,0.304969
4,YAL005C,1.270,1.2287,0.481748,0.135959
5,YAL007C,1.134,1.1995,-0.103126,0.105134
6,YAL008W,1.114,1.1079,-0.189137,0.008436


# Print out

In [35]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [36]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 17923092...
Inserting the new data...


100%|██████████| 2/2 [00:11<00:00,  5.96s/it]

Updating the data_modified_on field...
